In [1]:
!rm -rf models

In [2]:
from google.cloud import storage
from google.oauth2 import service_account

# Artifact Information
MODEL_ZIP_FILE = "models2023-03-27 20:03:08.zip"

# GCP Constants
BUCKET_NAME = "cloud-ai-platform-54bd831b-8d3c-431d-acbc-4f155b670016"
PROJECT_ID = "beaming-team-376517"
KEY_PATH = 'beaming-team-376517-82a98807d500.json'

# Deployment Parameters

REGION = "us-central1"
CLUSTER_NAME = "garbagenet-cluster"
IMAGE_NAME = "us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet"
IMAGE_TAG = "latest"

# GCP Set Up
def get_gcp_bucket():
    credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
    storage_client = storage.Client(project=PROJECT_ID,credentials=credentials)
    bucket = storage_client.bucket(BUCKET_NAME)
    return bucket

# Getting Model Artifact
def get_model(bucket):
    model_file_path = "Trained_models/" + MODEL_ZIP_FILE
    print(model_file_path)
    blob = bucket.blob(model_file_path)
    blob.download_to_filename('model.zip')
    
    
get_model(get_gcp_bucket())

Trained_models/models2023-03-27 20:03:08.zip


In [3]:
# Setting the required directory structure for tf-serving
!unzip 'model.zip'
!mkdir assets
!mv /home/jupyter/assets /home/jupyter/models/
!rm -rf "model.zip"

Archive:  model.zip
  inflating: models/fingerprint.pb   
  inflating: models/saved_model.pb   
   creating: models/assets/
  inflating: models/variables/variables.data-00000-of-00001  
  inflating: models/variables/variables.index  


In [4]:
%%writefile Dockerfile
# Docker pulls the specified image and sets it as the working image
ARG BASE_IMAGE="ubuntu:20.04"
FROM ${BASE_IMAGE}

# Allow log messages to be dumped in the stream (not buffer)
ENV PYTHONUNBUFFERED TRUE

# Install the Ubuntu dependencies and Python 3
RUN apt-get update \
    && apt-get install --no-install-recommends -y \
    ca-certificates \
    python3-dev \
    python3-distutils \
    python3-venv \
    curl \
    wget \
    unzip \
    gnupg \
    && rm -rf /var/lib/apt/lists/* \
    && cd /tmp \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py \
    && rm get-pip.py

# Create a new Python env and include it in the PATH
RUN python3 -m venv /home/venv
ENV PATH="/home/venv/bin:$PATH"

# Update the default system Python version to Python3/PIP3
RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1

# Creates a new Ubuntu user
RUN useradd -m model-server

# Upgrades PIP before proceeding and installs setuptools
RUN pip install pip --upgrade
RUN pip install -U pip setuptools

# Installs the TensorFlow Serving requirements
RUN pip install tensorflow-cpu==2.11.0
RUN echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
RUN apt-get update && apt-get install tensorflow-model-server -y

# Downloads the TensorFlow trained model
RUN cd /home \
    && mkdir /home/saved_models/ \
    && cd /home/saved_models/ \
    && mkdir garbagenet \
    && cd garbagenet \
    && mkdir dir 1

WORKDIR /home/saved_models/

COPY models/ /home/saved_models/garbagenet/1

RUN ls --recursive /home/saved_models/

RUN pwd \
    && ls -lart \
    && cd /home/saved_models/garbagenet \
    && ls -lart

# Sets the proper rights to the /home/saved_models dir and the created Python env
RUN chown -R model-server /home/saved_models \
    && chown -R model-server /home/venv
    
# Creates a directory for the logs and sets permissions to it
RUN mkdir /home/logs \
    && chown -R model-server /home/logs

# Defines the model_path environment variable and the model name
ENV MODEL_PATH=/home/saved_models/garbagenet
ENV MODEL_NAME=garbagenet

# Expose the ports 8500 (gRPC) and 8501 (REST)
ENV GRPC_PORT=8500
ENV REST_PORT=8501
EXPOSE $GRPC_PORT $REST_PORT

# Prepare the CMD that will be run on docker run
USER model-server
CMD tensorflow_model_server --port=$GRPC_PORT --rest_api_port=$REST_PORT --model_name=$MODEL_NAME --model_base_path=$MODEL_PATH >> /home/logs/server.log

Overwriting Dockerfile


In [5]:
%%writefile cloudbuild-build-image.yaml
steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet']
images:
- us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet

Overwriting cloudbuild-build-image.yaml


In [6]:
!gcloud config set project beaming-team-376517

Updated property [core/project].


In [7]:
!gcloud builds submit --config cloudbuild-build-image.yaml

Creating temporary tarball archive of 2623 file(s) totalling 297.7 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2023.03.28/03.06.11.451195.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://beaming-team-376517_cloudbuild/source/1679972771.577417-09a2ee9d5c73459c8f3a22f0f366127e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/beaming-team-376517/locations/global/builds/b3f85bfb-f7ac-44a8-ae13-5ef8a7ba35e1].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/b3f85bfb-f7ac-44a8-ae13-5ef8a7ba35e1?project=283526927149 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "b3f85bfb-f7ac-44a8-ae13-5ef8a7ba35e1"

FETCHSOURCE
Fetching storage object: gs://beaming-team-376517_cloudbuild/source/1679972771.577417-09a2ee

In [8]:
!gcloud container clusters create-auto {CLUSTER_NAME} \
    --region={REGION}

client [kubectl]. To install, run
  $ gcloud components install kubectl

Note: The Pod address range limits the maximum size of the cluster. Please refer to https://cloud.google.com/kubernetes-engine/docs/how-to/flexible-pod-cidr to learn how to optimize IP address allocation.
Creating cluster garbagenet-cluster in us-central1... Cluster is being configur
ed...⠼                                                                         
Creating cluster garbagenet-cluster in us-central1... Cluster is being deployed
...⠧                                                                           
Creating cluster garbagenet-cluster in us-central1... Cluster is being health-c
hecked (master is healthy)...done.                                             
Created [https://container.googleapis.com/v1/projects/beaming-team-376517/zones/us-central1/clusters/garbagenet-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1

In [9]:
%%writefile garbagenet.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: "garbagenet-deployment"
spec:
  selector:
    matchLabels:
      app: "garbagenet"
  replicas: 3
  template:
    metadata:
      labels:
        app: "garbagenet"
    spec:
      containers:
        - name: "garbagenet"
          image: us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet:latest
          ports:
          - containerPort: 8500
          - containerPort: 8501
---
apiVersion: "v1"
kind: "Service"
metadata:
  name: "garbagenet-service"
  namespace: "default"
  labels:
    app: "garbagenet"
spec:
  ports:
  - name: "tensorflow-svc"
    protocol: "TCP"
    port: 8500
    targetPort: 8500
  - name: "tensorflow-rest"
    protocol: "TCP"
    port: 8501
    targetPort: 8501
  selector:
    app: "garbagenet"
  type: "LoadBalancer"

Overwriting garbagenet.yaml


In [10]:
'''
%%writefile cloudbuild-deploy.yaml
steps:
- name: 'gcr.io/cloud-builders/gcloud'
  args:
  - 'container'
  - 'clusters'
  - 'get-credentials'
  - 'garbagenet-cluster'
  - '--region=us-central1'
  - '--project=beaming-team-376517'
- name: 'gcr.io/cloud-builders/kubectl'
  args:
  - 'apply'
  - '-f'
  - 'garbagenet.yaml'
'''

"\n%%writefile cloudbuild-deploy.yaml\nsteps:\n- name: 'gcr.io/cloud-builders/gcloud'\n  args:\n  - 'container'\n  - 'clusters'\n  - 'get-credentials'\n  - 'garbagenet-cluster'\n  - '--region=us-central1'\n  - '--project=beaming-team-376517'\n- name: 'gcr.io/cloud-builders/kubectl'\n  args:\n  - 'apply'\n  - '-f'\n  - 'garbagenet.yaml'\n"

In [11]:
%%writefile cloudbuild-deploy.yaml
steps:
- name: "gcr.io/cloud-builders/gke-deploy"
  args:
  - run
  - --filename=garbagenet.yaml
  - --image=us-central1-docker.pkg.dev/beaming-team-376517/garbagenet-tensorflow/garbagenet:latest
  - --location=us-central1
  - --cluster=garbagenet-cluster

Overwriting cloudbuild-deploy.yaml


In [12]:
!gcloud builds submit --config cloudbuild-deploy.yaml

Creating temporary tarball archive of 2775 file(s) totalling 298.8 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2023.03.28/03.22.09.966428.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://beaming-team-376517_cloudbuild/source/1679973730.102857-c0b4c67e17504e88b162f315cfca8dad.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/beaming-team-376517/locations/global/builds/4ef71af1-1551-4d34-b064-c01ea479836b].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/4ef71af1-1551-4d34-b064-c01ea479836b?project=283526927149 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "4ef71af1-1551-4d34-b064-c01ea479836b"

FETCHSOURCE
Fetching storage object: gs://beaming-team-376517_cloudbuild/source/1679973730.102857-c0b4c6